In [100]:
import pandas as pd
import networkx as nx
import numpy as np
import haversine as hs
import geopandas as gpd
import osmnx as ox

pd.set_option('display.max_rows', 20)

In [105]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_substations = pd.read_excel('./substations.xlsx', index_col=0)
#df_substations = pd.read_excel('./backup/substations_backup.xlsx', index_col=0)

In [106]:
for index, row in df_grid.iterrows():
    df_grid.loc[index, 'substation_1_name'] = df_substations.loc[row.substation_1, 'name']
    df_grid.loc[index, 'substation_2_name'] = df_substations.loc[row.substation_2, 'name']
    loc1=(df_substations.loc[row.substation_1, 'lat'], df_substations.loc[row.substation_1, 'lon'])
    loc2=(df_substations.loc[row.substation_2, 'lat'], df_substations.loc[row.substation_2, 'lon'])
    df_grid.loc[index, 'calculated_length'] = hs.haversine(loc1,loc2)

In [107]:
df_grid_line_disc = df_grid[(df_grid.calculated_length - df_grid.length > 50) & (df_grid.zone == 'FR')]
#df_grid_line_disc = df_grid[(2*df_grid.length < df_grid.calculated_length) & (df_grid.length > 20) & (df_grid.zone == 'FR')]
#df_grid_line_disc = df_grid[((df_grid.calculated_length - df_grid.length) > 50) & (df_grid.length > 20)]
display(df_grid_line_disc)

,name,eic,tso,substation_1,substation_2,voltage,resistance,reactance,susceptance,length,zone,substation_1_name,substation_2_name,calculated_length
1139,LIT 400kV N0 1 CHAFFARD (LE) - COULANGE,17T000000019997B,RTE,1340,1359,400,4.154,41.629,519.052,0.000,FR,CHAFFARD (LE),COULANGE,118.697547
1162,LIT 400kV N0 1 CHEVALET - WARANDE,17T-FR-000000139,RTE,145,1051,400,3.115,31.604,391.644,29.616,FR,CHEVALET,WARANDE,91.972369
1166,LIT 400kV N0 1 CLERAC - PLAUD,17T0000000207291,RTE,489,1401,400,3.972,44.447,429.556,40.341,FR,CLERAC,PLAUD,118.860821
1177,LIT 400kV N0 1 CRENEY - VIELMOULIN,17T000000021524E,RTE,21,1375,400,5.532,42.281,409.788,59.949,FR,CRENEY,VIELMOULIN,120.724648
1181,LIT 400kV N0 1 CUBNEZAIS - DONZAC,17T000000021765T,RTE,340,653,400,3.748,49.276,817.884,63.067,FR,CUBNEZAIS,DONZAC,146.478273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,LIT 400kV N0 2 GENISSIAT-POSTE-VIELMOULIN,17T000000024078X,RTE,938,1375,400,3.537,51.373,640.226,2.342,FR,GENISSIAT-POSTE,VIELMOULIN,164.880499
1370,LIT 400kV N0 2 HOUDREVILLE-MERY-SUR-SEINE,17T-FR-00000020C,RTE,451,1040,400,4.883,49.523,613.692,38.918,FR,HOUDREVILLE,MERY-SUR-SEINE,159.316924
1375,LIT 400kV N0 2 LONNY-MASTAING,17T-FR-000000236,RTE,291,549,400,2.374,31.112,515.832,10.831,FR,LONNY,MASTAING,105.362637
1381,LIT 400kV N0 2 MERY-SUR-SEINE-VIELMOULIN,17T000000026170Z,RTE,1040,1375,400,3.147,45.696,569.418,80.781,FR,MERY-SUR-SEINE,VIELMOULIN,144.614436


In [259]:
s1 = df_grid_line_disc.substation_1_name.unique()
s2 = df_grid_line_disc.substation_2_name.unique()
substations_disc = set(s1).union(s2)

In [204]:
df_missing_manual_completed = pd.read_excel('./missing_substations_completed.xlsx', index_col=0)

for index, row in df_missing_manual_completed.iterrows():
        if row.name in substations_disc:
            substations_disc.remove(row.name)
            
df_substations_disc = pd.DataFrame(list(zip(substations_disc)), columns=['name'])
len(substations_disc)

73

In [90]:
ox.settings.timeout = 3

for index, row in df_substations_disc.iterrows():
    try:
        tags = {
            'power_source': True,
            'substation': True,
            'electricity': True,
        }
        osm_find = ox.geometries_from_address(row['name'], tags)
        #osm_find = ox.overpass_request(query_data, pause=3, error_pause=3)
        if len(osm_find) > 0:
            print(osm_find.iloc[0])
            if hasattr(osm_find, "name"):
                df_substations_disc.loc[index, 'osm_find_name'] = osm_find.name[0]
            if hasattr(osm_find, "power"):
                df_substations_disc.loc[index, 'osm_find_type'] = osm_find.power[0]
            elif hasattr(osm_find, "substation"):
                df_substations_disc.loc[index, 'osm_find_type'] = osm_find.substation[0]
            if hasattr(osm_find, "operator"):
                df_substations_disc.loc[index, 'osm_find_operator'] = osm_find.operator[0]
            df_substations_disc.loc[index, 'lat'] = osm_find.geometry[0].representative_point().y
            df_substations_disc.loc[index, 'lon'] = osm_find.geometry[0].representative_point().x
        else:
            print('Not found: ', row['name'], ', ', index)
    except ValueError as ve:
        print('Value error, ', row['name'], ', ', index)
    except requests.exceptions.RequestException as rt:
        print('Timeout, ', row['name'], ', ', index)

access                                                             no
barrier                                                           NaN
geometry            POLYGON ((9.5141214 51.26011, 9.514762 51.2580...
nodes               [8778201949, 8778201950, 3700251846, 370025184...
name                                          Umspannwerk Bergshausen
note                                 Umspannwerk wird umgebaut 8/2014
operator                                                       TenneT
power                                                      substation
substation                                               transmission
voltage                                                        380000
addr:city                                                         NaN
addr:housename                                                    NaN
addr:housenumber                                                  NaN
addr:street                                                       NaN
frequency           

Not found:  Altentreptow/Nord ,  24
Not found:  Brinje ,  25
Not found:  Pestis  ,  26
power                                 substation
substation                    minor_distribution
geometry           POINT (18.6262064 54.3747858)
nodes                                        NaN
building                                     NaN
building:levels                              NaN
location                                     NaN
ref                                          NaN
voltage                                      NaN
height                                       NaN
street_cabinet                               NaN
Name: (node, 8576297006), dtype: object
nodes              [4429700862, 4429700864, 4429700865, 442970086...
building                                                         yes
building:levels                                                    1
location                                                      indoor
power                                                     

power                                                 substation
geometry       POLYGON ((14.618517 49.3330424, 14.6185351 49....
nodes          [2399154962, 6783574697, 2399154963, 239915496...
building                                       transformer_tower
location                                                  indoor
ref:ruian                                               43425747
source                                                     ruian
substation                                    minor_distribution
transformer                                         distribution
Name: (way, 231549152), dtype: object
barrier                                                           fence
geometry              POLYGON ((2.9360446 49.6576554, 2.9348745 49.6...
nodes                 [988793589, 988793445, 3716638336, 3716638483,...
location                                                        outdoor
name                                         Poste électrique de Laténa
operator         

Not found:  Gadalin ,  81
Not found:  Dumbrava ,  82
Not found:  Ens ,  83
nodes                 [9315550071, 9315550072, 9315550073, 931555007...
building                                                            yes
name                                              Poste de Fontcouverte
operator                                                    SNCF Réseau
operator:wikidata                                             Q21605526
operator:wikipedia                                       fr:SNCF Réseau
power                                                        substation
railway:position                                                739.993
substation                                                     traction
geometry              POLYGON ((4.8196819 43.9461626, 4.8197456 43.9...
Name: (way, 1009627875), dtype: object
access                                                             NaN
addr:city                                                          NaN
addr:housenumber        

Not found:  Retezat ,  103
location                                      indoor
operator                                         PRE
operator:wikidata                          Q15870812
operator:wikipedia             cs:Pražská energetika
power                                     substation
ref                                           TS8006
start_date                                1996-02-15
substation                        minor_distribution
transformer                             distribution
voltage:primary                                22000
voltage:secondary                                400
geometry               POINT (14.4309375 50.0885132)
barrier                                          NaN
source                                           NaN
note                                             NaN
layer                                            NaN
voltage                                          NaN
nodes                                            NaN
building           

...

Not found:  Lubmin ,  256
Not found:  Siroka ,  257
Value error,  VIELMOULIN ,  258
Not found:  Weingarten ,  259
Not found:  LANNEMEZAN ,  260
name                             ST UW Mecklar 1
power                                 substation
substation                    minor_distribution
geometry             POINT (9.748163 50.9179111)
nodes                                        NaN
access                                       NaN
addr:city                                    NaN
addr:country                                 NaN
addr:postcode                                NaN
addr:street                                  NaN
addr:suburb                                  NaN
frequency                                    NaN
operator                                     NaN
operator:wikidata                            NaN
voltage                                      NaN
barrier                                      NaN
industrial                                   NaN
landuse                

barrier                                      NaN
geometry             POINT (6.6018689 45.209109)
design:ref                                  PSSA
man_made                          street_cabinet
mapillary                        375891363759630
name                                  Le Villard
operator                                  Enedis
operator:wikidata                       Q3587594
power                                 substation
street_cabinet                             power
substation                    minor_distribution
survey:date                           2018-06-22
voltage                                20000;400
utility                                      NaN
source                                       NaN
nodes                                        NaN
building                                     NaN
location                                     NaN
ref:FR:RTE                                   NaN
ref:FR:RTE_nom                               NaN
layer               

geometry            POLYGON ((7.25259 52.9823289, 7.2526075 52.981...
nodes               [10220936435, 8328641499, 5398017784, 53980177...
addr:city                                                       Heede
addr:country                                                       DE
addr:housenumber                                                   14
addr:postcode                                                   26892
addr:state                                              Niedersachsen
addr:street                                               Deichstraße
fixme                         ungefähre Lage anhand Foto von Autobahn
frequency                                                        50;0
name                                                      Dörpen West
note                          Umspannanlage und DC-Einspeisung Dolwin
operator                                         TenneT Offshore GmbH
power                                                      substation
substation          

operator                                                         Enedis
operator:wikidata                                              Q3587594
power                                                        substation
source                Exploitants contributeurs de l'Agence ORE - 20...
substation                                           minor_distribution
voltage                                                           20000
geometry                                   POINT (2.2951351 51.0057993)
nodes                                                               NaN
building                                                            NaN
operator:wikipedia                                                  NaN
service                                                             NaN
source:power                                                        NaN
Name: (node, 9705196083), dtype: object
Not found:  RANCE-POSTE ,  345
geometry            POLYGON ((11.367394 48.2908127, 11.3673601 48

Not found:  J. Bohunice ,  364
barrier                                                       fence
geometry          POLYGON ((0.2402892 47.8843918, 0.2432114 47.8...
nodes             [999672280, 999672199, 999672154, 5368815720, ...
location                                                    outdoor
name                                   Poste électrique des Quintes
operator                                                        RTE
power                                                    substation
ref:FR:RTE                                                    QUINT
ref:FR:RTE_nom                                        QUINTES (LES)
substation                                             transmission
voltage                                                400000;22500
building                                                        NaN
service                                                         NaN
source                                                          NaN
source:power     

In [92]:
df_substations_try1 = df_substations_disc[(~df_substations_disc.lat.isnull()) & (~df_substations_disc.osm_find_name.isnull())]
display(df_substations_try1)
len(df_substations_try1)

,name,osm_find_name,osm_find_type,osm_find_operator,lat,lon
0,Bergshausen,Umspannwerk Bergshausen,substation,TenneT,51.258925,9.513845
5,Medzibrod,Medzibrod,substation,NaN,48.796483,19.366522
6,Remptendorf,Umspannwerk Remptendorf,substation,50Hertz Transmission,50.534476,11.646019
7,LOUISFERT,Poste électrique de Louisfert,substation,RTE,47.680194,-1.440791
12,FOSSES (LES),Tronjolly,substation,Enedis,48.107092,-1.681858
17,ZYDOWO KIERZKOWO,"Stacja elektroenergetyczna ""Żydowo"" 220/110 kV",substation,PSE; Energa Operator,54.024252,16.710177
20,OFFSHORE SWITCHYARD,OSY-OS1st Offshore Switchyard Substation,substation,NaN,51.582367,2.868583
21,COCHE (LA),Corpoelec,substation,CORPOELEC,10.452957,-66.921091
33,HAVRE (LE) (POSTE),EANA,substation,Enedis,49.537185,0.504254
36,Asslar,Umspannwerk Aßlar,substation,"TenneT, E.ON Mitte AG",50.591050,8.479999


88

In [93]:
df_substations_try1_2 = df_substations_try1.drop([12, 20, 21, 33, 56, 57, 67, 84, 94, 102, 105, 106, 113, 124, 131, 136, 196, 212, 231, 244, 289, 292, 359, 371])

In [94]:
display(df_substations_try1_2)

,name,osm_find_name,osm_find_type,osm_find_operator,lat,lon
0,Bergshausen,Umspannwerk Bergshausen,substation,TenneT,51.258925,9.513845
5,Medzibrod,Medzibrod,substation,NaN,48.796483,19.366522
6,Remptendorf,Umspannwerk Remptendorf,substation,50Hertz Transmission,50.534476,11.646019
7,LOUISFERT,Poste électrique de Louisfert,substation,RTE,47.680194,-1.440791
17,ZYDOWO KIERZKOWO,"Stacja elektroenergetyczna ""Żydowo"" 220/110 kV",substation,PSE; Energa Operator,54.024252,16.710177
36,Asslar,Umspannwerk Aßlar,substation,"TenneT, E.ON Mitte AG",50.591050,8.479999
52,BARJAC,Poste électrique de Barjac,substation,RTE,44.314844,4.343417
59,LATENA,Poste électrique de Laténa,substation,RTE,49.657862,2.938552
65,BARBUISE,Poste électrique de Barbuise,substation,RTE,48.561006,3.592026
72,Ragow,Umspannwerk Ragow,substation,50Hertz Transmission,51.875970,13.901369


In [103]:
for index, row in df_substations_try1_2.iterrows():
        i = df_substations.index[df_substations.name == row['name']].tolist()[0]
        df_substations.loc[i, 'lat'] = row['lat']
        df_substations.loc[i, 'lon'] = row['lon']
        
for index, row in df_grid.iterrows():
    loc1=(df_substations.loc[row.substation_1, 'lat'], df_substations.loc[row.substation_1, 'lon'])
    loc2=(df_substations.loc[row.substation_2, 'lat'], df_substations.loc[row.substation_2, 'lon'])
    df_grid.loc[index, 'calculated_length'] = hs.haversine(loc1,loc2)
    
df_grid_line_disc = df_grid[(1.5*df_grid.length < df_grid.calculated_length) & (df_grid.length > 10)]
display(df_grid_line_disc)

,name,eic,tso,substation_1,substation_2,voltage,resistance,reactance,susceptance,length,zone,substation_1_name,substation_2_name,calculated_length
58,Marke - Lauchstädt 504,11TD8L504------B,50HERTZ,378,28,400,2.104,12.729,230.153078,50.873000,DE,Marke,Lauchstädt,120.442741
186,Y-Ohlensehlen (-St. Huelfe - Wehrendorf) DUEMM...,11T0-0000-1115-Z,Amprion GmbH,545,1039,400,0.620,5.240,88.8,20.598875,DE,Y-St Huelfe,Ohlensehlen,33.559845
217,Y-Rommerskirchen (-Gohrpunkt - Osterath) FRIXH...,11T0-0000-1389-M,Amprion GmbH,1092,839,220,0.582,3.131,40.7,10.779786,DE,Rommerskirchen,Y-Osterath,28.496148
223,Daxlanden - Weingarten GERMHM S,11T-D4-D7-00003S,Amprion GmbH,810,612,400,0.444,4.088,71.4,16.314581,DE,Weingarten,Daxlanden,165.777175
252,Geithe - Unna HAARST W,11T0-0000-0607-E,Amprion GmbH,1293,1220,400,0.429,7.410,136,30.314467,DE,Geithe,Unna,96.888275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2617,Kuehmoos - Tiengen gr,11T-D4-D7-00012R,TRANSNETBW,1214,1327,220,0.670,3.530,81.36,16.191755,DE,Kuehmoos,Tiengen,24.399330
2619,Marbach - Pulverdingen gn,11T0-0000-0099-2,TRANSNETBW,1301,393,220,0.550,4.890,74.644,17.770000,DE,Marbach,Pulverdingen,213.742281
2630,Y-Daxlanden (-Buehl - Kuppenheim) rt,11T0-0000-1479-L,TRANSNETBW,100,612,220,1.160,5.710,77.47168,19.800000,DE,Y-Buehl,Daxlanden,36.594145
2643,Y-Kuehmoos (-Engstlatt - Villingen) gn,11TD400000587-2O,TRANSNETBW,309,1214,400,1.540,17.750,321.3535,68.000000,DE,Y-Engstlatt,Kuehmoos,104.673157


In [108]:
df_substations_none = df_substations_disc[df_substations_disc.lat.isnull()]
display(df_substations_none)

,name,osm_find_name,osm_find_type,osm_find_operator,lat,lon
1,Y-Doernigheim,NaN,NaN,NaN,NaN,NaN
2,Handewitt,NaN,NaN,NaN,NaN,NaN
3,Bergum,NaN,NaN,NaN,NaN,NaN
4,Bosaca,NaN,NaN,NaN,NaN,NaN
8,Podlog,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
363,Y-Dauersberg,NaN,NaN,NaN,NaN,NaN
364,J. Bohunice,NaN,NaN,NaN,NaN,NaN
369,YVELINES-OUEST,NaN,NaN,NaN,NaN,NaN
370,Weier,NaN,NaN,NaN,NaN,NaN


In [116]:
ox.settings.timeout = 5

for index, row in df_substations_none.iterrows():
    try:
        tags = {
            'substation': True,
        }
        osm_find = ox.geometries_from_address(row['name'], tags, dist=10000)
        if len(osm_find) > 0:
            print(osm_find.iloc[0])
            if hasattr(osm_find, "name"):
                df_substations_none.loc[index, 'osm_find_name'] = osm_find.name[0]
            if hasattr(osm_find, "power"):
                df_substations_none.loc[index, 'osm_find_type'] = osm_find.power[0]
            elif hasattr(osm_find, "substation"):
                df_substations_none.loc[index, 'osm_find_type'] = osm_find.substation[0]
            if hasattr(osm_find, "operator"):
                df_substations_none.loc[index, 'osm_find_operator'] = osm_find.operator[0]
            df_substations_none.loc[index, 'lat'] = osm_find.geometry[0].representative_point().y
            df_substations_none.loc[index, 'lon'] = osm_find.geometry[0].representative_point().x
        else:
            print('Not found: ', row['name'], ', ', index)
    except ValueError as ve:
        print('Value error, ', row['name'], ', ', index)
    except requests.exceptions.RequestException as rt:
        print('Timeout, ', row['name'], ', ', index)

barrier                                                  NaN
geometry                        POINT (8.8052872 50.0940804)
access                                                   NaN
location                                              indoor
power                                             substation
source:position         geschätzt nach Besichtigung mit bing
substation                                minor_distribution
name                                                     NaN
ref                                                      NaN
operator                                                 NaN
man_made                                                 NaN
street_cabinet                                           NaN
nodes                                                    NaN
voltage                                                  NaN
wikidata                                                 NaN
frequency                                                NaN
note                    

barrier                                       NaN
geometry             POINT (4.6790629 45.4172177)
location                                   indoor
name                                       Viaduc
operator                                   Enedis
operator:wikidata                        Q3587594
power                                  substation
source                           Enedis - 06/2018
substation                     minor_distribution
voltage                                 20000;400
transformer                                   NaN
source:date                                   NaN
nodes                                         NaN
ref:FR:RTE                                    NaN
ref:FR:RTE_nom                                NaN
building                                      NaN
service                                       NaN
source:power                                  NaN
wall                                          NaN
man_made                                      NaN


nodes                [2781915834, 2781915835, 2781915836, 278191583...
landuse                                                     industrial
name                                                         RP Brinje
operator                                                           HEP
operator:wikidata                                            Q13636760
power                                                       substation
substation                                                transmission
voltage                                                         220000
geometry             POLYGON ((15.1046684 44.9936868, 15.1061528 44...
Name: (way, 273326173), dtype: object
Not found:  Pestis  ,  26
barrier                                          NaN
geometry                POINT (13.2968388 51.008228)
source                                           NaN
location                                    platform
name                                    Bergmannsweg
power                           

Not found:  Stejaru ,  40
Value error,  Y-Kusenhorst ,  41
geometry             POINT (4.723317 49.768365)
name                                    Corneau
operator                                 Enedis
operator:wikidata                      Q3587594
power                                substation
substation                   minor_distribution
source                                      NaN
barrier                                     NaN
frequency                                   NaN
location                                    NaN
voltage                                     NaN
nodes                                       NaN
ref:FR:RTE                                  NaN
ref:FR:RTE_nom                              NaN
building                                    NaN
man_made                                    NaN
street_cabinet                              NaN
utility                                     NaN
source:power                                NaN
service                      

Not found:  Mariselu ,  55
Not found:  Marke ,  60
Not found:  Senj ,  61
barrier                                        NaN
geometry              POINT (2.6944113 47.8476712)
operator                                    Enedis
power                                   substation
source                            Enedis - 2021-01
substation                      minor_distribution
voltage                                      20000
nodes                                          NaN
cables                                         NaN
location                                       NaN
name                                           NaN
building                                       NaN
service                                        NaN
source:power                                   NaN
operator:wikidata                              NaN
operator:wikipedia                             NaN
ref:FR:RTE                                     NaN
ref:FR:RTE_nom                                 NaN
man_made

...

barrier                                       NaN
geometry             POINT (5.8365572 49.5631982)
location                                   indoor
operator                                     Ores
power                                  substation
ref                                        22-085
substation                     minor_distribution
voltage                                  6000;380
man_made                                      NaN
street_cabinet                                NaN
operator:wikidata                             NaN
utility                                       NaN
nodes                                         NaN
building                                      NaN
source                                        NaN
name                                          NaN
ref:FR:RTE                                    NaN
ref:FR:RTE_nom                                NaN
frequency                                     NaN
Name: (node, 6186289042), dtype: object
Not found:

nodes         [1127230598, 1127230973, 1127230387, 112723050...
name                                                    Ungheni
operator                                                    TEL
power                                                substation
substation                                         transmission
voltage                                           220000;110000
geometry      POLYGON ((24.4780723 46.4951109, 24.4790099 46...
building                                                    NaN
height                                                      NaN
Name: (way, 97334781), dtype: object
operator                                Enedis
power                               substation
source                        Enedis - 06/2018
substation                  minor_distribution
voltage                              20000;400
geometry          POINT (2.0093368 48.5333175)
nodes                                      NaN
building                                   NaN
service    

nodes         [1000877741, 1000877744, 1000877750, 100087775...
frequency                                                    60
location                                                outdoor
operator                                                  NYSEG
power                                                substation
substation                                         transmission
voltage                                                  115000
geometry      POLYGON ((-73.9383888 41.5644647, -73.9380455 ...
fenced                                                      NaN
barrier                                                     NaN
name                                                        NaN
Name: (way, 86220645), dtype: object
Not found:  Hamburg/Ost ,  328
addr:city                                Haarlem
addr:housenumber                              25
addr:street                     Van Merlenstraat
operator                                 Liander
operator:wikidata               

barrier                                        NaN
geometry              POINT (5.4220359 45.6845152)
operator                                    Enedis
power                                   substation
design:ref                                    PSSA
location                                     green
man_made                            street_cabinet
name                                       Château
ref:FR:gdo                              38297P0040
substation                      minor_distribution
transformer                           distribution
utility                                      power
voltage:primary                              20000
voltage:secondary                              400
voltage                                        NaN
source                                         NaN
street_cabinet                                 NaN
nodes                                          NaN
ref:FR:RTE                                     NaN
ref:FR:RTE_nom                 

building                                              transformer_tower
image                 https://www.flickr.com/photos/41449558@N06/667...
name                                                   Überführungshaus
object:suburb                                          Freusburgermühle
power                                                        substation
roof:shape                                                       hipped
substation                                           minor_distribution
geometry                                   POINT (7.8792174 50.8199252)
barrier                                                             NaN
man_made                                                            NaN
operator                                                            NaN
ref                                                                 NaN
street_cabinet                                                      NaN
nodes                                                           

In [119]:
display(df_substations_none)

,name,osm_find_name,osm_find_type,osm_find_operator,lat,lon
1,Y-Doernigheim,NaN,substation,NaN,50.094080,8.805287
2,Handewitt,Oeversee An der Treene,substation,Schleswig-Holstein Netz AG,54.685702,9.424470
3,Bergum,Ställverk Stenkullen,substation,NaN,57.802137,12.322050
4,Bosaca,NaN,substation,NaN,48.759700,17.887181
8,Podlog,RTP - Črnomelj,substation,NaN,45.572365,15.179460
9,Vetis,Vetiș,substation,TEL,47.773070,22.811324
10,Varin,NaN,NaN,NaN,49.138574,18.777688
11,CHEVALET,Viaduc,substation,Enedis,45.417218,4.679063
13,PLAUD,NaN,substation,RTE,45.608621,1.226971
14,SWIEBODZICE,Stacja elektroenergetyczna 220/110kV „Świebodz...,substation,TAURON Dystrybucja S.A.,50.854086,16.352273


In [120]:
df_substations_try2 = df_substations_none[(~df_substations_none.lat.isnull()) & (~df_substations_none.osm_find_name.isnull())]

In [122]:
len(df_substations_try2)

78